# TMDB API로 데이터 수집

## TMDB TV Series 데이터 전수 수집

1. 수집 대상: TMDB TV Series(드라마) 데이터
2. 수집 기간: 2016년 1월 1일 ~ 2025년 11월 29일 (약 11년치)
3. 수집 범위: 전체 페이지 (TMDB API 초당 요청 수(40 req/s) 제한, 500페이지 제한 등 주의)
4. 수집 결과: 53개 컬럼, 99320개 데이터, 약 144분 소요
5. 수집 방식: 동기 + 멀티스레드(ThreadPoolExecutor) 방식
5. 기타 사항: 결측치 보완 필요 -> episode_level 수집 시도했으나, 너무 많아서 제외 -> 시즌 정보 추가 수집 진행

In [1]:
import os
os.chdir(r"C:\Users\lizzy\스파르타_파이썬\tmdbapi실습")  # 원하는 경로로 변경
print(os.getcwd())  # 잘 바뀌었는지 확인

C:\Users\lizzy\스파르타_파이썬\tmdbapi실습


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # 현재 디렉터리의 .env 파일 읽어서 환경변수로 등록

API_KEY = os.getenv("API_KEY")  # .env에서 API 키 읽어오기

if API_KEY is None:
    raise ValueError("API_KEY가 설정되지 않았습니다. .env 파일에 API_KEY를 확인하세요.")

In [ ]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import time
import threading
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from dotenv import load_dotenv

# ==========================================================
# 0. API 키 및 기본 설정
# ==========================================================


BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}
TIMEOUT = 10
MAX_RETRIES = 3


# ==========================================================
# 1. Rate Limiter (초당 요청 수 제한: 25 req/s로 안정화)
# ==========================================================
class RateLimiter:
    """초당 요청 수를 제한하는 Rate Limiter (전역에서 공유)"""
    def __init__(self, max_calls=25, period=1.0):
        self.max_calls = max_calls
        self.period = period
        self.calls = []
        self.lock = threading.Lock()
        self.global_backoff_until = 0
    
    def wait(self):
        """필요시 대기 (429 또는 초당 요청 수 초과 방지)"""
        with self.lock:
            now = time.time()

            # 글로벌 백오프 체크
            if self.global_backoff_until > now:
                sleep_time = self.global_backoff_until - now
                print(f"⏸️  글로벌 백오프 대기 중... {sleep_time:.1f}초")
                time.sleep(sleep_time)
                now = time.time()

            # 오래된 호출 제거
            self.calls = [c for c in self.calls if now - c < self.period]
            
            if len(self.calls) >= self.max_calls:
                sleep_time = self.period - (now - self.calls[0]) + 0.05
                if sleep_time > 0:
                    time.sleep(sleep_time)
            
            self.calls.append(time.time())
    
    def set_global_backoff(self, seconds):
        """글로벌 백오프 설정 (429 에러 발생 시)"""
        with self.lock:
            backoff_until = time.time() + seconds
            if backoff_until > self.global_backoff_until:
                self.global_backoff_until = backoff_until
                print(f"🔴 글로벌 백오프 설정: {seconds}초")


# 전역 Rate Limiter (안정 25 req/s)
rate_limiter = RateLimiter(max_calls=25, period=1.0)


# ==========================================================
# 2. 세션 생성
# ==========================================================
def create_session():
    """재시도 로직과 연결 풀이 있는 세션 생성"""
    session = requests.Session()
    retry = Retry(
        total=5,
        backoff_factor=1,
        status_forcelist=[500, 502, 503, 504],
        respect_retry_after_header=True
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=20,
        pool_maxsize=20
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

session = create_session()


# ==========================================================
# 3. 공통 API 요청 함수 (tmdb_get)
# ==========================================================
def tmdb_get(path, params=None, retry_count=0):
    """
    TMDB GET 요청 공통 함수
    - api_key 자동 포함
    - Rate limit 처리
    - 429 / 타임아웃 / 기타 HTTP 에러 재시도
    """
    if retry_count >= MAX_RETRIES:
        print(f"⚠️  최대 재시도 횟수 초과: {path}")
        return None
    
    if params is None:
        params = {}
    
    params["api_key"] = API_KEY
    params.setdefault("language", "en-US")
    
    url = f"{BASE_URL}{path}"
    
    rate_limiter.wait()
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=TIMEOUT)
        
        # 429 에러 처리
        if response.status_code == 429:
            retry_after = response.headers.get('Retry-After')
            wait_time = int(retry_after) if retry_after else 10 * (2 ** retry_count)
            
            print(f"🔴 429 Rate Limit! {wait_time}초 대기... (재시도 {retry_count + 1}/{MAX_RETRIES})")
            rate_limiter.set_global_backoff(wait_time + 2)
            time.sleep(wait_time + 2)
            return tmdb_get(path, params, retry_count + 1)
        
        # 404 에러는 조용히 처리
        if response.status_code == 404:
            return None
        
        response.raise_for_status()
        return response.json()
        
    except requests.Timeout:
        print(f"⚠️  타임아웃: {path}")
        if retry_count < MAX_RETRIES:
            time.sleep(2 ** retry_count)
            return tmdb_get(path, params, retry_count + 1)
        return None
    except requests.RequestException as e:
        if retry_count < MAX_RETRIES:
            wait_time = 2 ** retry_count
            print(f"⚠️  요청 실패 (재시도 {retry_count + 1}/{MAX_RETRIES}): {e}")
            time.sleep(wait_time)
            return tmdb_get(path, params, retry_count + 1)
        print(f"❌ 최종 실패: {e}")
        return None


# ==========================================================
# 4. 헬퍼 함수
# ==========================================================
def simple_list_to_str(lst, key="name"):
    """리스트 안 dict에서 특정 key만 뽑아 문자열로 변환"""
    if not lst:
        return ""
    return ", ".join([str(item.get(key, "")) for item in lst if item.get(key)])


# ==========================================================
# 5. TV Series ID 수집 (100% 누락 방지: 재귀적 날짜 분할)
# ==========================================================
def fetch_single_page(page, start_date, end_date):
    """discover/tv 단일 페이지 수집"""
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": start_date,
        "first_air_date.lte": end_date,
        "page": page,
        "include_adult": True,
    }
    
    data = tmdb_get("/discover/tv", params)
    if data:
        return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)
    return [], 1, 0


def collect_ids_in_range(start_date, end_date, depth=0):
    """
    특정 날짜 구간에 대해:
    - total_pages <= 500: 해당 구간의 모든 페이지를 수집
    - total_pages > 500 : 날짜를 반으로 쪼개서 재귀적으로 수집
    → 500페이지 제한 때문에 ID가 잘리는 문제를 방지 (100% 누락 방지)
    """
    indent = "  " * depth
    results, total_pages, total_results = fetch_single_page(1, start_date, end_date)

    # 결과가 아예 없으면 조용히 종료
    if total_results == 0:
        print(f"{indent}📭 {start_date} ~ {end_date}: 결과 0개")
        return set()

    print(f"{indent}📅 {start_date} ~ {end_date}: 결과 {total_results:,}개, 페이지 {total_pages}개")

    # TMDB discover는 최대 500페이지만 접근 가능 → 500 이하이면 안전
    if total_pages <= 500:
        all_ids = set([s["id"] for s in results])
        max_pages = total_pages

        if max_pages > 1:
            # 나머지 페이지 멀티스레드 수집
            with ThreadPoolExecutor(max_workers=8) as executor:
                futures = {
                    executor.submit(fetch_single_page, page, start_date, end_date): page 
                    for page in range(2, max_pages + 1)
                }
                
                for future in tqdm(as_completed(futures), total=len(futures),
                                   desc=f"{indent}  페이지 수집", leave=False):
                    try:
                        page_results, _, _ = future.result()
                        for s in page_results:
                            all_ids.add(s["id"])
                    except Exception as e:
                        print(f"{indent}  페이지 수집 오류: {e}")

        print(f"{indent}✅ 확정 구간: {start_date} ~ {end_date} → ID {len(all_ids):,}개")
        return all_ids

    # total_pages > 500이면 날짜를 반으로 나누어 재귀 수집
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")

    if start_dt >= end_dt:
        # 이론상 거의 나오기 어려운 케이스지만, 방어적으로 메시지 출력
        print(f"{indent}⚠️ 단일일({start_date})에 500페이지 초과 발생. 일부 누락 가능.")
        # 여기서는 일단 500페이지만 수집 (완전 방지는 이론적으로 불가능 케이스)
        all_ids = set([s["id"] for s in results])
        max_pages = 500
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = {
                executor.submit(fetch_single_page, page, start_date, end_date): page 
                for page in range(2, max_pages + 1)
            }
            for future in tqdm(as_completed(futures), total=len(futures),
                               desc=f"{indent}  (비상) 페이지 수집", leave=False):
                try:
                    page_results, _, _ = future.result()
                    for s in page_results:
                        all_ids.add(s["id"])
                except Exception as e:
                    print(f"{indent}  페이지 수집 오류: {e}")
        return all_ids

    # 중간 날짜 계산해서 반으로 쪼개기
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid_date = mid_dt.date().strftime("%Y-%m-%d")

    # 두 구간: [start ~ mid], [mid+1 ~ end]
    left_end_dt = datetime.strptime(mid_date, "%Y-%m-%d")
    right_start_dt = left_end_dt + timedelta(days=1)

    left_start = start_date
    left_end = left_end_dt.strftime("%Y-%m-%d")
    right_start = right_start_dt.strftime("%Y-%m-%d")
    right_end = end_date

    print(f"{indent}🔀 페이지 500 초과 → 날짜 분할: [{left_start} ~ {left_end}], [{right_start} ~ {right_end}]")

    ids_left = collect_ids_in_range(left_start, left_end, depth+1)
    ids_right = collect_ids_in_range(right_start, right_end, depth+1)

    merged = ids_left.union(ids_right)
    print(f"{indent}✅ 병합: {start_date} ~ {end_date} → ID {len(merged):,}개")
    return merged


# ==========================================================
# 6. OTT 제공 정보 수집 (/tv/{id}/watch/providers 별도 호출)
# ==========================================================
def fetch_providers(series_id):
    """
    OTT 제공 정보 수집
    - /tv/{id}/watch/providers
    - 주요 국가: US, GB, KR
    """
    data = tmdb_get(f"/tv/{series_id}/watch/providers", params={"language": "en-US"})
    if not data:
        return ""
    
    results = data.get("results", {})
    all_providers = set()
    target_countries = ['US', 'GB', 'KR']
    
    for country in target_countries:
        if country in results:
            info = results[country]
            flatrate = info.get("flatrate", [])
            for p in flatrate:
                name = p.get("provider_name")
                if name:
                    all_providers.add(name)
    
    return ", ".join(sorted(all_providers)) if all_providers else ""


# ==========================================================
# 7. TV Series 상세 정보 수집 (append_to_response + 별도 providers)
# ==========================================================
def fetch_tv_details(series_id):
    """
    TV 시리즈 상세 정보 수집
    - /tv/{id}?append_to_response=reviews,keywords,aggregate_credits
    - watch/providers는 별도 엔드포인트에서 수집
    """
    params = {
        "append_to_response": "reviews,keywords,aggregate_credits"
    }
    
    data = tmdb_get(f"/tv/{series_id}", params)
    if not data:
        return None

    try:
        # last_episode_to_air
        last_ep = data.get("last_episode_to_air") or {}
        
        # genre_ids / genres
        genres_list = data.get("genres", []) or []
        genre_ids = ", ".join([str(g.get("id", "")) for g in genres_list if g.get("id") is not None])

        created_by_str = simple_list_to_str(data.get("created_by", []))

        # ===== 기본 필드 =====
        record = {
            "id": data.get("id"),
            "title": data.get("name"),
            "type": "tv_series",
            "adult": data.get("adult"),
            "backdrop_path": data.get("backdrop_path"),
            "created_by": created_by_str,
            "episode_run_time": ", ".join(map(str, data.get("episode_run_time", []))) if data.get("episode_run_time") else "",
            "first_air_date": data.get("first_air_date"),
            "genres": simple_list_to_str(genres_list),
            "genre_ids": genre_ids,
            "homepage": data.get("homepage"),
            "in_production": data.get("in_production"),
            "languages": ", ".join(data.get("languages", [])) if data.get("languages") else "",
            "last_air_date": data.get("last_air_date"),

            # last_episode_to_air 세부
            "last_episode_to_air_id": last_ep.get("id"),
            "last_episode_to_air_name": last_ep.get("name"),
            "last_episode_to_air_overview": last_ep.get("overview"),
            "last_episode_to_air_vote_average": last_ep.get("vote_average"),
            "last_episode_to_air_vote_count": last_ep.get("vote_count"),
            "last_episode_to_air_air_date": last_ep.get("air_date"),
            "last_episode_to_air_episode_number": last_ep.get("episode_number"),
            "last_episode_to_air_production_code": last_ep.get("production_code"),
            "last_episode_to_air_runtime": last_ep.get("runtime"),
            "last_episode_to_air_season_number": last_ep.get("season_number"),
            "last_episode_to_air_show_id": last_ep.get("show_id"),
            "last_episode_to_air_still_path": last_ep.get("still_path"),
            
            "name": data.get("name"),
            "next_episode_to_air": str(data.get("next_episode_to_air")) if data.get("next_episode_to_air") else "",
            "networks": simple_list_to_str(data.get("networks", [])),
            "number_of_episodes": data.get("number_of_episodes"),
            "number_of_seasons": data.get("number_of_seasons"),
            "origin_country": ", ".join(data.get("origin_country", [])) if data.get("origin_country") else "",
            "original_language": data.get("original_language"),
            "original_name": data.get("original_name"),
            "overview": data.get("overview"),
            "popularity": data.get("popularity"),
            "poster_path": data.get("poster_path"),
            "production_companies": simple_list_to_str(data.get("production_companies", [])),
            "production_countries": simple_list_to_str(data.get("production_countries", [])),
            "seasons": "; ".join(
                f"S{s.get('season_number')}: {s.get('name')} ({s.get('episode_count')} eps)"
                for s in (data.get("seasons") or [])
                if s.get('season_number') is not None
            ),
            "spoken_languages": simple_list_to_str(data.get("spoken_languages", [])),
            "status": data.get("status"),
            "tagline": data.get("tagline"),
            "type_detail": data.get("type"),
            "vote_average": data.get("vote_average"),
            "vote_count": data.get("vote_count"),
        }

        # ===== 리뷰 (reviews) =====
        reviews_block = data.get("reviews") or {}
        reviews = reviews_block.get("results", []) if isinstance(reviews_block, dict) else []
        all_reviews = []
        for review in reviews[:5]:
            author = review.get("author", "unknown")
            rating = review.get("author_details", {}).get("rating")
            rating_str = f" (Rating: {rating})" if rating is not None else ""
            content = review.get("content", "").replace("\n", " ").replace("\r", " ")[:200]
            all_reviews.append(f"{author}{rating_str}: {content}")
        record["review"] = " || ".join(all_reviews) if all_reviews else ""

        # ===== 키워드 (keywords) =====
        keywords_block = data.get("keywords") or {}
        keywords = keywords_block.get("results", []) if isinstance(keywords_block, dict) else []
        record["keyword"] = ", ".join([k.get("name", "") for k in keywords[:20] if k.get("name")])

        # ===== aggregate_credits (top_cast / directors / writers) =====
        credits = data.get("aggregate_credits") or {}
        cast = credits.get("cast", []) if isinstance(credits, dict) else []
        crew = credits.get("crew", []) if isinstance(credits, dict) else []

        # 주연 배우 4명
        record["top_cast"] = ", ".join([c.get("name", "") for c in cast[:4] if c.get("name")])

        directors_set = set()
        writers_set = set()
        for c in crew:
            name = c.get("name", "")
            if not name:
                continue
            jobs = c.get("jobs", []) or []
            for job in jobs:
                job_name = job.get("job", "") or ""
                if "Director" in job_name or "Series Director" in job_name:
                    directors_set.add(name)
                if job_name in ["Writer", "Screenplay", "Story", "Writer's Assistant"]:
                    writers_set.add(name)

        record["directors"] = ", ".join(sorted(directors_set)[:10])
        record["writers"] = ", ".join(sorted(writers_set)[:10])

        # ===== providers (별도 엔드포인트 호출) =====
        record["providers"] = fetch_providers(series_id)

        return record

    except Exception as e:
        print(f"상세 정보 처리 오류 (ID: {series_id}): {e}")
        return None


# ==========================================================
# 8. MAIN 실행부
# ==========================================================
if __name__ == "__main__":
    print("=" * 90)
    print("🚀 TMDB 드라마 전체 수집 (2016-01-01 ~ 2025-11-29)")
    print("=" * 90)

    start_date = "2005-01-01"
    end_date = "2015-12-31"
    total_start = time.time()

    print(f"\n📅 전체 기간: {start_date} ~ {end_date}")
    print(f"⚡ Rate Limit: 초당 25 요청 (TMDB 제한 40 대비 여유)")
    print(f"⚠️ discover/tv 500페이지 제한 → 재귀적 날짜 분할로 100% 누락 방지")
    print("="*90)

    # 1) ID 전체 수집 (재귀 분할)
    print("\n📌 1단계: TV 시리즈 ID 전체 수집 시작")
    all_tv_ids = collect_ids_in_range(start_date, end_date)

    if not all_tv_ids:
        print("\n⚠️ 수집된 ID가 없습니다. 종료합니다.")
        raise SystemExit

    tv_list = sorted(list(all_tv_ids))
    print(f"\n{'='*90}")
    print(f"✨ 총 고유 ID: {len(tv_list):,}개")

    # 2) 상세 정보 수집
    print(f"\n📌 2단계: 상세 정보 수집 시작...")
    print(f"⚡ Workers: 6개 (안전한 동시성)")
    print(f"⚡ Rate Limit: 초당 25 요청")
    print(f"⏱  대략적인 최소 이론치: {len(tv_list) / 25 / 60:.1f}분 (ID당 1~2회 호출 포함)")
    print(f"💾 자동 백업: 매 1000개 또는 5분마다")
    print("="*90 + "\n")

    results = []
    failed_ids = []
    last_save_time = time.time()
    save_interval = 300  # 5분

    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = {executor.submit(fetch_tv_details, sid): sid for sid in tv_list}

        for future in tqdm(as_completed(futures), total=len(futures), desc="TV 상세 수집"):
            series_id = futures[future]
            try:
                detail = future.result()
                if detail:
                    results.append(detail)
                else:
                    failed_ids.append(series_id)
                
                # 중간 저장
                current_time = time.time()
                if (len(results) % 1000 == 0 and len(results) > 0) or \
                   (current_time - last_save_time > save_interval and len(results) > 0):
                    df_temp = pd.DataFrame(results)
                    backup_file = f"backup_{len(results)}_{datetime.now().strftime('%H%M')}.csv"
                    df_temp.to_csv(backup_file, index=False, encoding="utf-8-sig")
                    print(f"\n💾 백업 저장: {backup_file} ({len(results):,}개)")
                    last_save_time = current_time
                    
            except Exception as e:
                print(f"\n수집 오류 (ID: {series_id}): {e}")
                failed_ids.append(series_id)

    print(f"\n🎉 상세 정보 수집 완료: {len(results):,}개")
    if failed_ids:
        print(f"⚠️  실패한 ID: {len(failed_ids)}개")
        with open("failed_ids.txt", "w") as f:
            f.write("\n".join(map(str, failed_ids)))

    # 3) 최종 저장
    if results:
        df = pd.DataFrame(results)

        # 정렬
        if "first_air_date" in df.columns and "popularity" in df.columns:
            df = df.sort_values(["first_air_date", "popularity"], ascending=[True, False])

        # 53번째 컬럼: "created by" (공백 버전) 추가
        if "created_by" in df.columns:
            df["created by"] = df["created_by"]

        # 컬럼 순서 정리 (요청 스펙 53개)
        column_order = [
            "id",
            "title",
            "type",
            "adult",
            "backdrop_path",
            "created_by",
            "episode_run_time",
            "first_air_date",
            "genres",
            "genre_ids",
            "homepage",
            "in_production",
            "languages",
            "last_air_date",
            "last_episode_to_air_id",
            "last_episode_to_air_name",
            "last_episode_to_air_overview",
            "last_episode_to_air_vote_average",
            "last_episode_to_air_vote_count",
            "last_episode_to_air_air_date",
            "last_episode_to_air_episode_number",
            "last_episode_to_air_production_code",
            "last_episode_to_air_runtime",
            "last_episode_to_air_season_number",
            "last_episode_to_air_show_id",
            "last_episode_to_air_still_path",
            "name",
            "next_episode_to_air",
            "networks",
            "number_of_episodes",
            "number_of_seasons",
            "origin_country",
            "original_language",
            "original_name",
            "overview",
            "popularity",
            "poster_path",
            "production_companies",
            "production_countries",
            "seasons",
            "spoken_languages",
            "status",
            "tagline",
            "type_detail",
            "vote_average",
            "vote_count",
            "review",
            "keyword",
            "top_cast",
            "directors",
            "writers",
            "created by",     # alias
            "providers",
        ]
        
        existing_columns = [col for col in column_order if col in df.columns]
        df = df[existing_columns]
        
        output_file = "tv_series_2016_2025_FULL.csv"
        df.to_csv(output_file, index=False, encoding="utf-8-sig")

        elapsed = time.time() - total_start

        print("\n" + "="*90)
        print("========================== DONE ==========================")
        print(f"📌 총 데이터: {len(df):,}개")
        print(f"📊 컬럼 수: {len(df.columns)}개 (요청 53개 반영)")
        print(f"⏱  소요시간: {elapsed/60:.2f}분 ({elapsed/3600:.2f}시간)")
        print(f"💾 저장 파일: {output_file}")
        print(f"📈 평균 처리 속도: {len(df)/(elapsed/60):.1f}개/분")
        if failed_ids:
            print(f"⚠️  실패한 ID: {len(failed_ids)}개 (failed_ids.txt 참조)")
        
        # 간단 요약
        if "first_air_date" in df.columns:
            print("\n[데이터 요약]")
            print(f"- 연도 범위: {df['first_air_date'].min()} ~ {df['first_air_date'].max()}")
        if "popularity" in df.columns:
            print(f"- 평균 인기도: {df['popularity'].mean():.2f}")
        if "vote_average" in df.columns:
            print(f"- 평균 평점: {df['vote_average'].mean():.2f}")
        if "number_of_episodes" in df.columns:
            print(f"- 총 에피소드: {df['number_of_episodes'].sum():,.0f}개")

        print("="*90)
    else:
        print("\n⚠️ 수집된 상세 데이터가 없습니다.")


🚀 TMDB 드라마 전체 수집 (2016-01-01 ~ 2025-11-29)

📅 전체 기간: 2005-01-01 ~ 2015-12-31
⚡ Rate Limit: 초당 25 요청 (TMDB 제한 40 대비 여유)
⚠️ discover/tv 500페이지 제한 → 재귀적 날짜 분할로 100% 누락 방지

📌 1단계: TV 시리즈 ID 전체 수집 시작
📅 2005-01-01 ~ 2015-12-31: 결과 43,869개, 페이지 2194개
🔀 페이지 500 초과 → 날짜 분할: [2005-01-01 ~ 2010-07-02], [2010-07-03 ~ 2015-12-31]
  📅 2005-01-01 ~ 2010-07-02: 결과 17,936개, 페이지 897개
  🔀 페이지 500 초과 → 날짜 분할: [2005-01-01 ~ 2007-10-02], [2007-10-03 ~ 2010-07-02]
    📅 2005-01-01 ~ 2007-10-02: 결과 8,086개, 페이지 405개


    ✅ 확정 구간: 2005-01-01 ~ 2007-10-02 → ID 8,086개
    📅 2007-10-03 ~ 2010-07-02: 결과 9,850개, 페이지 493개


    ✅ 확정 구간: 2007-10-03 ~ 2010-07-02 → ID 9,850개
  ✅ 병합: 2005-01-01 ~ 2010-07-02 → ID 17,936개
  📅 2010-07-03 ~ 2015-12-31: 결과 25,933개, 페이지 1297개
  🔀 페이지 500 초과 → 날짜 분할: [2010-07-03 ~ 2013-04-01], [2013-04-02 ~ 2015-12-31]
    📅 2010-07-03 ~ 2013-04-01: 결과 11,856개, 페이지 593개
    🔀 페이지 500 초과 → 날짜 분할: [2010-07-03 ~ 2011-11-16], [2011-11-17 ~ 2013-04-01]
      📅 2010-07-03 ~ 2011-11-16: 결과 5,767개, 페이지 289개


      ✅ 확정 구간: 2010-07-03 ~ 2011-11-16 → ID 5,767개
      📅 2011-11-17 ~ 2013-04-01: 결과 6,089개, 페이지 305개


      ✅ 확정 구간: 2011-11-17 ~ 2013-04-01 → ID 6,089개
    ✅ 병합: 2010-07-03 ~ 2013-04-01 → ID 11,856개
    📅 2013-04-02 ~ 2015-12-31: 결과 14,077개, 페이지 704개
    🔀 페이지 500 초과 → 날짜 분할: [2013-04-02 ~ 2014-08-16], [2014-08-17 ~ 2015-12-31]
      📅 2013-04-02 ~ 2014-08-16: 결과 6,499개, 페이지 325개


      ✅ 확정 구간: 2013-04-02 ~ 2014-08-16 → ID 6,499개
      📅 2014-08-17 ~ 2015-12-31: 결과 7,578개, 페이지 379개


      ✅ 확정 구간: 2014-08-17 ~ 2015-12-31 → ID 7,578개
    ✅ 병합: 2013-04-02 ~ 2015-12-31 → ID 14,077개
  ✅ 병합: 2010-07-03 ~ 2015-12-31 → ID 25,933개
✅ 병합: 2005-01-01 ~ 2015-12-31 → ID 43,869개

✨ 총 고유 ID: 43,869개

📌 2단계: 상세 정보 수집 시작...
⚡ Workers: 6개 (안전한 동시성)
⚡ Rate Limit: 초당 25 요청
⏱  대략적인 최소 이론치: 29.2분 (ID당 1~2회 호출 포함)
💾 자동 백업: 매 1000개 또는 5분마다



TV 상세 수집:   2%|█                                                             | 765/43869 [01:06<1:02:14, 11.54it/s]


In [ ]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import time
import threading
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ==========================================================
# 0. API 키 및 기본 설정
# ==========================================================


BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}
TIMEOUT = 10
MAX_RETRIES = 3


# ==========================================================
# 1. Rate Limiter (초당 요청 수 제한: 25 req/s로 안정화)
# ==========================================================
class RateLimiter:
    """초당 요청 수를 제한하는 Rate Limiter (전역에서 공유)"""
    def __init__(self, max_calls=25, period=1.0):
        self.max_calls = max_calls
        self.period = period
        self.calls = []
        self.lock = threading.Lock()
        self.global_backoff_until = 0
    
    def wait(self):
        """필요시 대기 (429 또는 초당 요청 수 초과 방지)"""
        with self.lock:
            now = time.time()

            # 글로벌 백오프 체크
            if self.global_backoff_until > now:
                sleep_time = self.global_backoff_until - now
                print(f"⏸️  글로벌 백오프 대기 중... {sleep_time:.1f}초")
                time.sleep(sleep_time)
                now = time.time()

            # 오래된 호출 제거
            self.calls = [c for c in self.calls if now - c < self.period]
            
            if len(self.calls) >= self.max_calls:
                sleep_time = self.period - (now - self.calls[0]) + 0.05
                if sleep_time > 0:
                    time.sleep(sleep_time)
            
            self.calls.append(time.time())
    
    def set_global_backoff(self, seconds):
        """글로벌 백오프 설정 (429 에러 발생 시)"""
        with self.lock:
            backoff_until = time.time() + seconds
            if backoff_until > self.global_backoff_until:
                self.global_backoff_until = backoff_until
                print(f"🔴 글로벌 백오프 설정: {seconds}초")


# 전역 Rate Limiter (안정 25 req/s)
rate_limiter = RateLimiter(max_calls=25, period=1.0)


# ==========================================================
# 2. 세션 생성
# ==========================================================
def create_session():
    """재시도 로직과 연결 풀이 있는 세션 생성"""
    session = requests.Session()
    retry = Retry(
        total=5,
        backoff_factor=1,
        status_forcelist=[500, 502, 503, 504],
        respect_retry_after_header=True
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=20,
        pool_maxsize=20
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

session = create_session()


# ==========================================================
# 3. 공통 API 요청 함수 (tmdb_get)
# ==========================================================
def tmdb_get(path, params=None, retry_count=0):
    """
    TMDB GET 요청 공통 함수
    - api_key 자동 포함
    - Rate limit 처리
    - 429 / 타임아웃 / 기타 HTTP 에러 재시도
    """
    if retry_count >= MAX_RETRIES:
        print(f"⚠️  최대 재시도 횟수 초과: {path}")
        return None
    
    if params is None:
        params = {}
    
    params["api_key"] = API_KEY
    params.setdefault("language", "en-US")
    
    url = f"{BASE_URL}{path}"
    
    rate_limiter.wait()
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=TIMEOUT)
        
        # 429 에러 처리
        if response.status_code == 429:
            retry_after = response.headers.get('Retry-After')
            wait_time = int(retry_after) if retry_after else 10 * (2 ** retry_count)
            
            print(f"🔴 429 Rate Limit! {wait_time}초 대기... (재시도 {retry_count + 1}/{MAX_RETRIES})")
            rate_limiter.set_global_backoff(wait_time + 2)
            time.sleep(wait_time + 2)
            return tmdb_get(path, params, retry_count + 1)
        
        # 404 에러는 조용히 처리
        if response.status_code == 404:
            return None
        
        response.raise_for_status()
        return response.json()
        
    except requests.Timeout:
        print(f"⚠️  타임아웃: {path}")
        if retry_count < MAX_RETRIES:
            time.sleep(2 ** retry_count)
            return tmdb_get(path, params, retry_count + 1)
        return None
    except requests.RequestException as e:
        if retry_count < MAX_RETRIES:
            wait_time = 2 ** retry_count
            print(f"⚠️  요청 실패 (재시도 {retry_count + 1}/{MAX_RETRIES}): {e}")
            time.sleep(wait_time)
            return tmdb_get(path, params, retry_count + 1)
        print(f"❌ 최종 실패: {e}")
        return None


# ==========================================================
# 4. 헬퍼 함수
# ==========================================================
def simple_list_to_str(lst, key="name"):
    """리스트 안 dict에서 특정 key만 뽑아 문자열로 변환"""
    if not lst:
        return ""
    return ", ".join([str(item.get(key, "")) for item in lst if item.get(key)])


# ==========================================================
# 5. TV Series ID 수집 (100% 누락 방지: 재귀적 날짜 분할)
# ==========================================================
def fetch_single_page(page, start_date, end_date):
    """discover/tv 단일 페이지 수집"""
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": start_date,
        "first_air_date.lte": end_date,
        "page": page,
        "include_adult": True,
    }
    
    data = tmdb_get("/discover/tv", params)
    if data:
        return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)
    return [], 1, 0


def collect_ids_in_range(start_date, end_date, depth=0):
    """
    특정 날짜 구간에 대해:
    - total_pages <= 500: 해당 구간의 모든 페이지를 수집
    - total_pages > 500 : 날짜를 반으로 쪼개서 재귀적으로 수집
    → 500페이지 제한 때문에 ID가 잘리는 문제를 방지 (100% 누락 방지)
    """
    indent = "  " * depth
    results, total_pages, total_results = fetch_single_page(1, start_date, end_date)

    # 결과가 아예 없으면 조용히 종료
    if total_results == 0:
        print(f"{indent}📭 {start_date} ~ {end_date}: 결과 0개")
        return set()

    print(f"{indent}📅 {start_date} ~ {end_date}: 결과 {total_results:,}개, 페이지 {total_pages}개")

    # TMDB discover는 최대 500페이지만 접근 가능 → 500 이하이면 안전
    if total_pages <= 500:
        all_ids = set([s["id"] for s in results])
        max_pages = total_pages

        if max_pages > 1:
            # 나머지 페이지 멀티스레드 수집
            with ThreadPoolExecutor(max_workers=8) as executor:
                futures = {
                    executor.submit(fetch_single_page, page, start_date, end_date): page 
                    for page in range(2, max_pages + 1)
                }
                
                for future in tqdm(as_completed(futures), total=len(futures),
                                   desc=f"{indent}  페이지 수집", leave=False):
                    try:
                        page_results, _, _ = future.result()
                        for s in page_results:
                            all_ids.add(s["id"])
                    except Exception as e:
                        print(f"{indent}  페이지 수집 오류: {e}")

        print(f"{indent}✅ 확정 구간: {start_date} ~ {end_date} → ID {len(all_ids):,}개")
        return all_ids

    # total_pages > 500이면 날짜를 반으로 나누어 재귀 수집
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")

    if start_dt >= end_dt:
        # 이론상 거의 나오기 어려운 케이스지만, 방어적으로 메시지 출력
        print(f"{indent}⚠️ 단일일({start_date})에 500페이지 초과 발생. 일부 누락 가능.")
        # 여기서는 일단 500페이지만 수집 (완전 방지는 이론적으로 불가능 케이스)
        all_ids = set([s["id"] for s in results])
        max_pages = 500
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = {
                executor.submit(fetch_single_page, page, start_date, end_date): page 
                for page in range(2, max_pages + 1)
            }
            for future in tqdm(as_completed(futures), total=len(futures),
                               desc=f"{indent}  (비상) 페이지 수집", leave=False):
                try:
                    page_results, _, _ = future.result()
                    for s in page_results:
                        all_ids.add(s["id"])
                except Exception as e:
                    print(f"{indent}  페이지 수집 오류: {e}")
        return all_ids

    # 중간 날짜 계산해서 반으로 쪼개기
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid_date = mid_dt.date().strftime("%Y-%m-%d")

    # 두 구간: [start ~ mid], [mid+1 ~ end]
    left_end_dt = datetime.strptime(mid_date, "%Y-%m-%d")
    right_start_dt = left_end_dt + timedelta(days=1)

    left_start = start_date
    left_end = left_end_dt.strftime("%Y-%m-%d")
    right_start = right_start_dt.strftime("%Y-%m-%d")
    right_end = end_date

    print(f"{indent}🔀 페이지 500 초과 → 날짜 분할: [{left_start} ~ {left_end}], [{right_start} ~ {right_end}]")

    ids_left = collect_ids_in_range(left_start, left_end, depth+1)
    ids_right = collect_ids_in_range(right_start, right_end, depth+1)

    merged = ids_left.union(ids_right)
    print(f"{indent}✅ 병합: {start_date} ~ {end_date} → ID {len(merged):,}개")
    return merged


# ==========================================================
# 6. OTT 제공 정보 수집 (/tv/{id}/watch/providers 별도 호출)
# ==========================================================
def fetch_providers(series_id):
    """
    OTT 제공 정보 수집
    - /tv/{id}/watch/providers
    - 주요 국가: US, GB, KR
    """
    data = tmdb_get(f"/tv/{series_id}/watch/providers", params={"language": "en-US"})
    if not data:
        return ""
    
    results = data.get("results", {})
    all_providers = set()
    target_countries = ['US', 'GB', 'KR']
    
    for country in target_countries:
        if country in results:
            info = results[country]
            flatrate = info.get("flatrate", [])
            for p in flatrate:
                name = p.get("provider_name")
                if name:
                    all_providers.add(name)
    
    return ", ".join(sorted(all_providers)) if all_providers else ""


# ==========================================================
# 7. TV Series 상세 정보 수집 (append_to_response + 별도 providers)
# ==========================================================
def fetch_tv_details(series_id):
    """
    TV 시리즈 상세 정보 수집
    - /tv/{id}?append_to_response=reviews,keywords,aggregate_credits
    - watch/providers는 별도 엔드포인트에서 수집
    """
    params = {
        "append_to_response": "reviews,keywords,aggregate_credits"
    }
    
    data = tmdb_get(f"/tv/{series_id}", params)
    if not data:
        return None

    try:
        # last_episode_to_air
        last_ep = data.get("last_episode_to_air") or {}
        
        # genre_ids / genres
        genres_list = data.get("genres", []) or []
        genre_ids = ", ".join([str(g.get("id", "")) for g in genres_list if g.get("id") is not None])

        created_by_str = simple_list_to_str(data.get("created_by", []))

        # ===== 기본 필드 (name, created by, poster_path 제외) =====
        record = {
            "id": data.get("id"),
            "title": data.get("name"),
            "type": "tv_series",
            "adult": data.get("adult"),
            "backdrop_path": data.get("backdrop_path"),
            "created_by": created_by_str,
            "episode_run_time": ", ".join(map(str, data.get("episode_run_time", []))) if data.get("episode_run_time") else "",
            "first_air_date": data.get("first_air_date"),
            "genres": simple_list_to_str(genres_list),
            "genre_ids": genre_ids,
            "homepage": data.get("homepage"),
            "in_production": data.get("in_production"),
            "languages": ", ".join(data.get("languages", [])) if data.get("languages") else "",
            "last_air_date": data.get("last_air_date"),

            # last_episode_to_air 세부
            "last_episode_to_air_id": last_ep.get("id"),
            "last_episode_to_air_name": last_ep.get("name"),
            "last_episode_to_air_overview": last_ep.get("overview"),
            "last_episode_to_air_vote_average": last_ep.get("vote_average"),
            "last_episode_to_air_vote_count": last_ep.get("vote_count"),
            "last_episode_to_air_air_date": last_ep.get("air_date"),
            "last_episode_to_air_episode_number": last_ep.get("episode_number"),
            "last_episode_to_air_production_code": last_ep.get("production_code"),
            "last_episode_to_air_runtime": last_ep.get("runtime"),
            "last_episode_to_air_season_number": last_ep.get("season_number"),
            "last_episode_to_air_show_id": last_ep.get("show_id"),
            "last_episode_to_air_still_path": last_ep.get("still_path"),
            
            "next_episode_to_air": str(data.get("next_episode_to_air")) if data.get("next_episode_to_air") else "",
            "networks": simple_list_to_str(data.get("networks", [])),
            "number_of_episodes": data.get("number_of_episodes"),
            "number_of_seasons": data.get("number_of_seasons"),
            "origin_country": ", ".join(data.get("origin_country", [])) if data.get("origin_country") else "",
            "original_language": data.get("original_language"),
            "original_name": data.get("original_name"),
            "overview": data.get("overview"),
            "popularity": data.get("popularity"),
            "production_companies": simple_list_to_str(data.get("production_companies", [])),
            "production_countries": simple_list_to_str(data.get("production_countries", [])),
            "seasons": "; ".join(
                f"S{s.get('season_number')}: {s.get('name')} ({s.get('episode_count')} eps)"
                for s in (data.get("seasons") or [])
                if s.get('season_number') is not None
            ),
            "spoken_languages": simple_list_to_str(data.get("spoken_languages", [])),
            "status": data.get("status"),
            "tagline": data.get("tagline"),
            "type_detail": data.get("type"),
            "vote_average": data.get("vote_average"),
            "vote_count": data.get("vote_count"),
        }

        # ===== 리뷰 (reviews) =====
        reviews_block = data.get("reviews") or {}
        reviews = reviews_block.get("results", []) if isinstance(reviews_block, dict) else []
        all_reviews = []
        for review in reviews[:5]:
            author = review.get("author", "unknown")
            rating = review.get("author_details", {}).get("rating")
            rating_str = f" (Rating: {rating})" if rating is not None else ""
            content = review.get("content", "").replace("\n", " ").replace("\r", " ")[:200]
            all_reviews.append(f"{author}{rating_str}: {content}")
        record["review"] = " || ".join(all_reviews) if all_reviews else ""

        # ===== 키워드 (keywords) =====
        keywords_block = data.get("keywords") or {}
        keywords = keywords_block.get("results", []) if isinstance(keywords_block, dict) else []
        record["keyword"] = ", ".join([k.get("name", "") for k in keywords[:20] if k.get("name")])

        # ===== aggregate_credits (top_cast / directors / writers) =====
        credits = data.get("aggregate_credits") or {}
        cast = credits.get("cast", []) if isinstance(credits, dict) else []
        crew = credits.get("crew", []) if isinstance(credits, dict) else []

        # 주연 배우 4명
        record["top_cast"] = ", ".join([c.get("name", "") for c in cast[:4] if c.get("name")])

        directors_set = set()
        writers_set = set()
        for c in crew:
            name = c.get("name", "")
            if not name:
                continue
            jobs = c.get("jobs", []) or []
            for job in jobs:
                job_name = job.get("job", "") or ""
                if "Director" in job_name or "Series Director" in job_name:
                    directors_set.add(name)
                if job_name in ["Writer", "Screenplay", "Story", "Writer's Assistant"]:
                    writers_set.add(name)

        record["directors"] = ", ".join(sorted(directors_set)[:10])
        record["writers"] = ", ".join(sorted(writers_set)[:10])

        # ===== providers (별도 엔드포인트 호출) =====
        record["providers"] = fetch_providers(series_id)

        return record

    except Exception as e:
        print(f"상세 정보 처리 오류 (ID: {series_id}): {e}")
        return None


# ==========================================================
# 8. MAIN 실행부
# ==========================================================
if __name__ == "__main__":
    print("=" * 90)
    print("🚀 TMDB 드라마 전체 수집 (2005-01-01 ~ 2015-12-31)")
    print("=" * 90)

    start_date = "2005-01-01"
    end_date = "2015-12-31"
    total_start = time.time()

    print(f"\n📅 전체 기간: {start_date} ~ {end_date}")
    print(f"⚡ Rate Limit: 초당 25 요청 (TMDB 제한 40 대비 여유)")
    print(f"⚠️ discover/tv 500페이지 제한 → 재귀적 날짜 분할로 100% 누락 방지")
    print("="*90)

    # 1) ID 전체 수집 (재귀 분할)
    print("\n📌 1단계: TV 시리즈 ID 전체 수집 시작")
    all_tv_ids = collect_ids_in_range(start_date, end_date)

    if not all_tv_ids:
        print("\n⚠️ 수집된 ID가 없습니다. 종료합니다.")
        raise SystemExit

    tv_list = sorted(list(all_tv_ids))
    print(f"\n{'='*90}")
    print(f"✨ 총 고유 ID: {len(tv_list):,}개")

    # 2) 상세 정보 수집
    print(f"\n📌 2단계: 상세 정보 수집 시작...")
    print(f"⚡ Workers: 6개 (안전한 동시성)")
    print(f"⚡ Rate Limit: 초당 25 요청")
    print(f"⏱  대략적인 최소 이론치: {len(tv_list) / 25 / 60:.1f}분 (ID당 1~2회 호출 포함)")
    print(f"💾 자동 백업: 매 1000개 또는 5분마다")
    print("="*90 + "\n")

    results = []
    failed_ids = []
    last_save_time = time.time()
    save_interval = 300  # 5분

    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = {executor.submit(fetch_tv_details, sid): sid for sid in tv_list}

        for future in tqdm(as_completed(futures), total=len(futures), desc="TV 상세 수집"):
            series_id = futures[future]
            try:
                detail = future.result()
                if detail:
                    results.append(detail)
                else:
                    failed_ids.append(series_id)
                
                # 중간 저장
                current_time = time.time()
                if (len(results) % 1000 == 0 and len(results) > 0) or \
                   (current_time - last_save_time > save_interval and len(results) > 0):
                    df_temp = pd.DataFrame(results)
                    backup_file = f"backup_{len(results)}_{datetime.now().strftime('%H%M')}.csv"
                    df_temp.to_csv(backup_file, index=False, encoding="utf-8-sig")
                    print(f"\n💾 백업 저장: {backup_file} ({len(results):,}개)")
                    last_save_time = current_time
                    
            except Exception as e:
                print(f"\n수집 오류 (ID: {series_id}): {e}")
                failed_ids.append(series_id)

    print(f"\n🎉 상세 정보 수집 완료: {len(results):,}개")
    if failed_ids:
        print(f"⚠️  실패한 ID: {len(failed_ids)}개")
        with open("failed_ids.txt", "w") as f:
            f.write("\n".join(map(str, failed_ids)))

    # 3) 최종 저장
    if results:
        df = pd.DataFrame(results)

        # 정렬
        if "first_air_date" in df.columns and "popularity" in df.columns:
            df = df.sort_values(["first_air_date", "popularity"], ascending=[True, False])

        # 컬럼 순서 정리 (name, created by, poster_path 제외)
        column_order = [
            "id",
            "title",
            "type",
            "adult",
            "backdrop_path",
            "created_by",
            "episode_run_time",
            "first_air_date",
            "genres",
            "genre_ids",
            "homepage",
            "in_production",
            "languages",
            "last_air_date",
            "last_episode_to_air_id",
            "last_episode_to_air_name",
            "last_episode_to_air_overview",
            "last_episode_to_air_vote_average",
            "last_episode_to_air_vote_count",
            "last_episode_to_air_air_date",
            "last_episode_to_air_episode_number",
            "last_episode_to_air_production_code",
            "last_episode_to_air_runtime",
            "last_episode_to_air_season_number",
            "last_episode_to_air_show_id",
            "last_episode_to_air_still_path",
            "next_episode_to_air",
            "networks",
            "number_of_episodes",
            "number_of_seasons",
            "origin_country",
            "original_language",
            "original_name",
            "overview",
            "popularity",
            "production_companies",
            "production_countries",
            "seasons",
            "spoken_languages",
            "status",
            "tagline",
            "type_detail",
            "vote_average",
            "vote_count",
            "review",
            "keyword",
            "top_cast",
            "directors",
            "writers",
            "providers",
        ]
        
        existing_columns = [col for col in column_order if col in df.columns]
        df = df[existing_columns]
        
        output_file = "tv_series_2005_2015_FULL.csv"
        df.to_csv(output_file, index=False, encoding="utf-8-sig")

        elapsed = time.time() - total_start

        print("\n" + "="*90)
        print("========================== DONE ==========================")
        print(f"📌 총 데이터: {len(df):,}개")
        print(f"📊 컬럼 수: {len(df.columns)}개")
        print(f"⏱  소요시간: {elapsed/60:.2f}분 ({elapsed/3600:.2f}시간)")
        print(f"💾 저장 파일: {output_file}")
        print(f"📈 평균 처리 속도: {len(df)/(elapsed/60):.1f}개/분")
        if failed_ids:
            print(f"⚠️  실패한 ID: {len(failed_ids)}개 (failed_ids.txt 참조)")
        
        # 간단 요약
        if "first_air_date" in df.columns:
            print("\n[데이터 요약]")
            print(f"- 연도 범위: {df['first_air_date'].min()} ~ {df['first_air_date'].max()}")
        if "popularity" in df.columns:
            print(f"- 평균 인기도: {df['popularity'].mean():.2f}")
        if "vote_average" in df.columns:
            print(f"- 평균 평점: {df['vote_average'].mean():.2f}")
        if "number_of_episodes" in df.columns:
            print(f"- 총 에피소드: {df['number_of_episodes'].sum():,.0f}개")

        print("="*90)
    else:
        print("\n⚠️ 수집된 상세 데이터가 없습니다.")

🚀 TMDB 드라마 전체 수집 (2005-01-01 ~ 2015-12-31)

📅 전체 기간: 2005-01-01 ~ 2015-12-31
⚡ Rate Limit: 초당 25 요청 (TMDB 제한 40 대비 여유)
⚠️ discover/tv 500페이지 제한 → 재귀적 날짜 분할로 100% 누락 방지

📌 1단계: TV 시리즈 ID 전체 수집 시작
📅 2005-01-01 ~ 2015-12-31: 결과 43,869개, 페이지 2194개
🔀 페이지 500 초과 → 날짜 분할: [2005-01-01 ~ 2010-07-02], [2010-07-03 ~ 2015-12-31]
  📅 2005-01-01 ~ 2010-07-02: 결과 17,936개, 페이지 897개
  🔀 페이지 500 초과 → 날짜 분할: [2005-01-01 ~ 2007-10-02], [2007-10-03 ~ 2010-07-02]
    📅 2005-01-01 ~ 2007-10-02: 결과 8,086개, 페이지 405개


    ✅ 확정 구간: 2005-01-01 ~ 2007-10-02 → ID 8,086개
    📅 2007-10-03 ~ 2010-07-02: 결과 9,850개, 페이지 493개


    ✅ 확정 구간: 2007-10-03 ~ 2010-07-02 → ID 9,850개
  ✅ 병합: 2005-01-01 ~ 2010-07-02 → ID 17,936개
  📅 2010-07-03 ~ 2015-12-31: 결과 25,933개, 페이지 1297개
  🔀 페이지 500 초과 → 날짜 분할: [2010-07-03 ~ 2013-04-01], [2013-04-02 ~ 2015-12-31]
    📅 2010-07-03 ~ 2013-04-01: 결과 11,856개, 페이지 593개
    🔀 페이지 500 초과 → 날짜 분할: [2010-07-03 ~ 2011-11-16], [2011-11-17 ~ 2013-04-01]
      📅 2010-07-03 ~ 2011-11-16: 결과 5,767개, 페이지 289개


      ✅ 확정 구간: 2010-07-03 ~ 2011-11-16 → ID 5,767개
      📅 2011-11-17 ~ 2013-04-01: 결과 6,089개, 페이지 305개


      ✅ 확정 구간: 2011-11-17 ~ 2013-04-01 → ID 6,089개
    ✅ 병합: 2010-07-03 ~ 2013-04-01 → ID 11,856개
    📅 2013-04-02 ~ 2015-12-31: 결과 14,077개, 페이지 704개
    🔀 페이지 500 초과 → 날짜 분할: [2013-04-02 ~ 2014-08-16], [2014-08-17 ~ 2015-12-31]
      📅 2013-04-02 ~ 2014-08-16: 결과 6,499개, 페이지 325개


...

TV 상세 수집:  66%|█████████████████████████████████████████                     | 29027/43869 [52:16<16:37, 14.89it/s]


💾 백업 저장: backup_29000_1551.csv (29,000개)


TV 상세 수집:  68%|██████████████████████████████████████████▍                   | 30028/43869 [53:42<15:34, 14.81it/s]


💾 백업 저장: backup_30000_1553.csv (30,000개)


TV 상세 수집:  69%|███████████████████████████████████████████                   | 30434/43869 [54:17<23:58,  9.34it/s]


## 수집 결과 점검

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('tv_series_2016_2025_FULL.csv')

print("="*80)
print("📌 TMDB TV DATASET VALIDATION REPORT")
print("="*80)

# -------------------------------------------------------------------
# 1) 컬럼 53개 존재 여부 체크
# -------------------------------------------------------------------
expected_cols = [
    "id", "title", "type", "adult", "backdrop_path", "created_by",
    "episode_run_time", "first_air_date", "genres", "genre_ids",
    "homepage", "in_production", "languages", "last_air_date",
    "last_episode_to_air_id", "last_episode_to_air_name",
    "last_episode_to_air_overview", "last_episode_to_air_vote_average",
    "last_episode_to_air_vote_count", "last_episode_to_air_air_date",
    "last_episode_to_air_episode_number", "last_episode_to_air_production_code",
    "last_episode_to_air_runtime", "last_episode_to_air_season_number",
    "last_episode_to_air_show_id", "last_episode_to_air_still_path",
    "name", "next_episode_to_air", "networks", "number_of_episodes",
    "number_of_seasons", "origin_country", "original_language",
    "original_name", "overview", "popularity", "poster_path",
    "production_companies", "production_countries", "seasons",
    "spoken_languages", "status", "tagline", "type_detail",
    "vote_average", "vote_count", "review", "keyword",
    "top_cast", "directors", "writers", "providers"
]

missing = [c for c in expected_cols if c not in df.columns]
extra = [c for c in df.columns if c not in expected_cols]

print("\n[1] 컬럼 검증")
print(f"- 총 컬럼 수: {len(df.columns)}")
print(f"- 누락된 컬럼: {missing}")
print(f"- 추가된(예상 외) 컬럼: {extra}")

# -------------------------------------------------------------------
# 2) 결측치 비율 체크
# -------------------------------------------------------------------
print("\n[2] 결측치 비율 TOP 20")
na_rate = df.isna().mean().sort_values(ascending=False)
print(na_rate.head(20))

# -------------------------------------------------------------------
# 3) ID 중복 체크
# -------------------------------------------------------------------
print("\n[3] ID 중복 체크")
dup = df["id"].duplicated().sum()
print(f"- 중복 ID 수: {dup}")

# -------------------------------------------------------------------
# 4) 날짜 컬럼 검증
# -------------------------------------------------------------------
date_cols = ["first_air_date", "last_air_date", "last_episode_to_air_air_date"]

print("\n[4] 날짜형 컬럼 검증")
for col in date_cols:
    try:
        parsed = pd.to_datetime(df[col], errors="coerce")
        nulls = parsed.isna().sum()
        print(f"- {col}: 파싱 실패 {nulls}개")
    except Exception as e:
        print(f"- {col}: 오류 발생 {e}")

# -------------------------------------------------------------------
# 5) 숫자형 컬럼 범위 검증
# -------------------------------------------------------------------
num_cols = ["popularity", "vote_average", "vote_count",
            "number_of_episodes", "number_of_seasons"]

print("\n[5] 숫자형 컬럼 범위 검증")
for col in num_cols:
    if col in df.columns:
        print(f"\n[{col}]")
        print(df[col].describe())

# -------------------------------------------------------------------
# 6) 기초 통계 검증
# -------------------------------------------------------------------
print("\n[6] 전체 통계 요약")
print(df.describe(include="all").transpose().head(15))

# -------------------------------------------------------------------
# 7) 문자열 주요 필드 검증
# -------------------------------------------------------------------
string_cols = ["genres", "keyword", "providers", "top_cast"]

print("\n[7] 문자열 필드 간단 검증")
for col in string_cols:
    uniq = df[col].nunique(dropna=True)
    empty = (df[col] == "").sum() if col in df else None
    print(f"- {col}: 고유값 {uniq}개, 빈 문자열 수 {empty}")

print("\n🎉 데이터 검증 완료!")
print("="*80)

📌 TMDB TV DATASET VALIDATION REPORT

[1] 컬럼 검증
- 총 컬럼 수: 53
- 누락된 컬럼: []
- 추가된(예상 외) 컬럼: ['created by']

[2] 결측치 비율 TOP 20
last_episode_to_air_production_code    0.997795
next_episode_to_air                    0.987767
review                                 0.981474
tagline                                0.949235
writers                                0.803776
providers                              0.787324
created by                             0.781041
created_by                             0.781041
keyword                                0.695671
last_episode_to_air_still_path         0.694201
last_episode_to_air_overview           0.679752
directors                              0.675362
episode_run_time                       0.622211
homepage                               0.593949
production_companies                   0.563794
last_episode_to_air_runtime            0.495258
overview                               0.442207
production_countries                   0.436881
networks     